In [ ]:
import numpy as np
import datashader as ds
import datashader.transfer_functions as tf
from datashader.colors import inferno, viridis
from datashader.transfer_functions import set_background
from colorcet import palette
import pandas as pd
import matplotlib.pyplot as plt

import holoviews as hv
from colorcet import fire
import dask.dataframe as dd
from colorcet import fire
from holoviews.operation.datashader import datashade
hv.extension('bokeh', 'matplotlib')
from holoviews import opts

# References
# https://casyfill.github.io/posts/2016/08/big_data_mapping/
# https://aetperf.github.io/2020/02/13/Lunch-break,-plotting-traffic-injuries-with-datashader.html

# Process data

In [ ]:
df = pd.read_parquet('data/geolife_trajectories_1_3.parquet')
print(df.shape[0])
df.head()

# Make a video with a zooming into the city

In [ ]:
# lat zoom from range 10, 60 > 39.8, 40.1
# lon zoom from range 70,140 > 116.2, 116.5

#    x_range=(116.2,116.5),
#    y_range=(39.8, 40.1),

# Define zoom
lat_range_start = (10.0, 60.0)
lat_range_stop  = (39.8, 40.1)

lon_range_start = (70.0,  140.0)
lon_range_stop  = (116.2, 116.5)

static_frames = 10
zoom_steps    = 150

west_lat_steps  = np.linspace(lat_range_start[0], lat_range_stop[0], zoom_steps)
east_lon_steps  = np.linspace(lon_range_start[0], lon_range_stop[0], zoom_steps)
north_lon_steps = np.linspace(lon_range_start[1], lon_range_stop[1], zoom_steps)
south_lat_steps = np.linspace(lat_range_start[1], lat_range_stop[1], zoom_steps)

x_range = zip(west_lat_steps, east_lon_steps)
y_range = zip(south_lat_steps, north_lon_steps)

# Plot settings
height_to_width = 2
width = 800
# This scale is used to remove very bright spots as you zoom
max_count = np.linspace(8000, 200, zoom_steps)
images = []

# Static view of all of all the data
cvs = ds.Canvas(plot_width=width, plot_height=int(width*height_to_width))
agg = cvs.points(df, 'lon', 'lat')
agg_values = np.where(agg>max_count[0], max_count[0], agg.values)
agg.values = agg_values
img = tf.shade(agg,  cmap=palette["fire"], how='log')
img = set_background(img, 'black')

frame = 0
# Generate a bunch of static images for the full map
for itt in range(static_frames):
    filename = f"data/movie_0_static_{itt}"
    #filename = "data/movie_%s_%s_%s" % (f"{frame:04}", "_".join([f"{x:.2f}" for x in x_range]), "_".join([f"{y:.2f}" for y in y_range]))
    ds.utils.export_image(img, filename, fmt=".png")
    frame += 1

for fig_no, (x_rng, y_rng) in enumerate(zip(x_range, y_range)):
    # Zoom towards the target
    cvs = ds.Canvas(plot_width=width, plot_height=int(width*height_to_width), x_range=x_rng, y_range=y_rng)
    agg = cvs.points(df, 'lon', 'lat')

    # Scale values
    agg_values = np.where(agg>max_count[fig_no], max_count[fig_no], agg.values)
    agg.values = agg_values

    img = tf.shade(agg,  cmap=palette["fire"], how='log')
    img = set_background(img, 'black')
    # Save image if you want to view it later
    images.append(img)

    filename = f"data/movie_1_zoom_{fig_no}"
    #filename = "data/movie_%s_%s_%s" % (f"{frame:04}", "_".join([f"{x:.2f}" for x in x_rng]), "_".join([f"{y:.2f}" for y in y_rng]))
    ds.utils.export_image(img, filename, fmt=".png")
    frame += 1

for fig_no_extra in range(static_frames):
    # Write the last image a couple of extra times to stall the video on the final frame
    filename = f"data/movie_2_final_{fig_no_extra}"
    #filename = "/data/movie_%s_%s_%s" % (f"{frame:04}", "_".join([f"{x:.2f}" for x in x_rng]), "_".join([f"{y:.2f}" for y in y_rng]))
    ds.utils.export_image(img, filename, fmt=".png")
    frame += 1

# Show the last image

In [ ]:
img

# Create a movie from the images

First you need to join the images into a movie using the ffmpeg tool in Linux

```bash
ffmpeg -y -r 25 -f image2 -pattern_type glob -i 'data/movie_*.png' -vcodec libx264  -crf 25 -pix_fmt yuv420p /data/movie.mp4
```
This will create a mp4 video in /tmp with 25 frames per second.

